In [1]:
import pandas as pd
import numpy as np
import os
import requests
import json
from datetime import datetime

In [2]:
# The following code efficiently reads and concatenates all CSV files from a specified folder.
# It uses a generator expression within pd.concat(), which is memory-efficient as it
# reads one file at a time rather than loading all files into memory at once.
# This is the most effective way to handle multiple large data files with the same structure.

# Define the path to the folder containing the data files.
folderpath = r"data" # The 'r' prefix handles path strings correctly.

# Create a list of the full file paths for all CSV files in the folder.
filepaths = [os.path.join(folderpath, name) for name in os.listdir(folderpath) if name.endswith('.csv')]

# Concatenate all data files into a single DataFrame.
# We add the dtype parameter to specify that columns 5 and 7 should be read as strings.
# This prevents pandas from guessing and resolves the DtypeWarning.
df_citi_bike = pd.concat((pd.read_csv(f, dtype={5: str, 7: str}) for f in filepaths), ignore_index=True)

print("All 12 months of Citi Bike data have been successfully loaded and joined.")
print(f"Total trips in 2022: {len(df_citi_bike)}")
display(df_citi_bike.head())

All 12 months of Citi Bike data have been successfully loaded and joined.
Total trips in 2022: 29838806


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,9D0DC440CB40CF8E,electric_bike,2022-08-27 13:56:47.728,2022-08-27 14:02:56.651,Flatbush Ave & Ocean Ave,3704.04,3 St & Prospect Park West,3865.05,40.663657,-73.963014,40.668132,-73.973638,casual
1,2214991DFBE5C4D7,electric_bike,2022-08-20 10:37:02.756,2022-08-20 10:45:56.631,Forsyth St\t& Grand St,5382.07,E 11 St & 1 Ave,5746.14,40.717798,-73.993161,40.729538,-73.984267,casual
2,20C5D469563B6337,classic_bike,2022-08-31 18:55:03.051,2022-08-31 19:03:37.344,Perry St & Bleecker St,5922.07,Grand St & Greene St,5500.02,40.735354,-74.004831,40.721700,-74.002381,member
3,3E8791885BC189D1,classic_bike,2022-08-02 08:05:00.250,2022-08-02 08:16:52.063,FDR Drive & E 35 St,6230.04,Grand Army Plaza & Central Park S,6839.10,40.744219,-73.971212,40.764397,-73.973715,member
4,8DBCBF98885106CB,electric_bike,2022-08-25 15:44:48.386,2022-08-25 15:55:39.691,E 40 St & 5 Ave,6474.11,Ave A & E 14 St,5779.11,40.752052,-73.982115,40.730311,-73.980472,member


In [7]:
# For this exercise, we will place private token here. In a real project, use environment variables.
NOAA_TOKEN = 'mxNJXmYdcuisvlaPKEtXzNVfVJjqasRi' # <-- PASTE TOKEN HERE

# Define the API endpoint and parameters for LaGuardia Airport for 2022.
# Station ID for LaGuardia Airport: GHCND:USW00014732
# Dataset ID: GHCND (Global Historical Climatology Network Daily)
# Datatype: TAVG (Average Temperature)
# Limit: 1000 (to ensure all 365 days are captured)
API_URL = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00014732&startdate=2022-01-01&enddate=2022-12-31"

In [4]:
# Make the API request with the token in the headers for authentication.
try:
    r = requests.get(API_URL, headers={'token': NOAA_TOKEN})
    r.raise_for_status() # This will raise an error for bad status codes (4xx or 5xx)
    print("Successfully fetched data from NOAA API.")
    
    # Load the JSON response.
    d = json.loads(r.text)
    
    # Extract the relevant data using list comprehensions.
    dates_temp = [item['date'] for item in d['results']]
    temps = [item['value'] for item in d['results']]
    
    # Create a pandas DataFrame from the lists.
    df_temps = pd.DataFrame()
    
    # Convert date strings to datetime objects.
    df_temps['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
    
    # The temperature is in tenths of a degree Celsius, so divide by 10.
    df_temps['avgTemp'] = [float(v) / 10.0 for v in temps]
    
    # Save the weather data to a CSV for future use.
    df_temps.to_csv('nyc_weather_2022.csv', index=False)
    print("Weather data processed and saved to 'nyc_weather_2022.csv'.")
    display(df_temps.head())

except requests.exceptions.HTTPError as err:
    print(f"HTTP Error occurred: {err}")
    print("Please check your NOAA Token and the API URL.")
except Exception as e:
    print(f"An error occurred: {e}")

Successfully fetched data from NOAA API.
Weather data processed and saved to 'nyc_weather_2022.csv'.


,date,avgTemp
0,2022-01-01,11.6
1,2022-01-02,11.4
2,2022-01-03,1.4
3,2022-01-04,-2.7
4,2022-01-05,3.2


In [5]:
# To merge the two dataframes, we need a common key. We will use the date.
# First, ensure the date columns in both dataframes are in the same format.

# Convert the 'started_at' column in the bike data to datetime objects.
df_citi_bike['started_at'] = pd.to_datetime(df_citi_bike['started_at'])

# Create a 'date' column in the bike data by extracting just the date part.
df_citi_bike['date'] = df_citi_bike['started_at'].dt.date

# The weather data 'date' column is already a datetime object. We need to convert it too.
df_temps['date'] = pd.to_datetime(df_temps['date']).dt.date

# Now, merge the two DataFrames on the 'date' column.
# A 'left' merge ensures we keep all bike trip records.
df_merged = pd.merge(df_citi_bike, df_temps, on='date', how='left', indicator=True)

# Save the final, enriched dataset to a new CSV file.
# Note: This will be a very large file.
df_merged.to_csv('citi_bike_2022_with_weather.csv', index=False)

print("Bike data and weather data successfully merged.")
print(f"Total records in merged data: {len(df_merged)}")
display(df_merged.head())

Bike data and weather data successfully merged.
Total records in merged data: 29838806


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date,avgTemp,_merge
0,9D0DC440CB40CF8E,electric_bike,2022-08-27 13:56:47.728,2022-08-27 14:02:56.651,Flatbush Ave & Ocean Ave,3704.04,3 St & Prospect Park West,3865.05,40.663657,-73.963014,40.668132,-73.973638,casual,2022-08-27,27.8,both
1,2214991DFBE5C4D7,electric_bike,2022-08-20 10:37:02.756,2022-08-20 10:45:56.631,Forsyth St\t& Grand St,5382.07,E 11 St & 1 Ave,5746.14,40.717798,-73.993161,40.729538,-73.984267,casual,2022-08-20,27.9,both
2,20C5D469563B6337,classic_bike,2022-08-31 18:55:03.051,2022-08-31 19:03:37.344,Perry St & Bleecker St,5922.07,Grand St & Greene St,5500.02,40.735354,-74.004831,40.721700,-74.002381,member,2022-08-31,25.6,both
3,3E8791885BC189D1,classic_bike,2022-08-02 08:05:00.250,2022-08-02 08:16:52.063,FDR Drive & E 35 St,6230.04,Grand Army Plaza & Central Park S,6839.10,40.744219,-73.971212,40.764397,-73.973715,member,2022-08-02,26.4,both
4,8DBCBF98885106CB,electric_bike,2022-08-25 15:44:48.386,2022-08-25 15:55:39.691,E 40 St & 5 Ave,6474.11,Ave A & E 14 St,5779.11,40.752052,-73.982115,40.730311,-73.980472,member,2022-08-25,28.1,both


In [6]:
# Check the merge results
df_merged['_merge'].value_counts()

_merge
both          29838166
left_only          640
right_only           0
Name: count, dtype: int64